# Képek betöltése Mnist adatbázisból

In [19]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Képek átméretezése train és test szétosztása

In [20]:
x_train = x_train / 255
x_test = x_test / 255

x_train = x_train.reshape(-1,28,28,1)
x_test = x_test.reshape(-1,28,28,1)
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

batch_size=30

# Alap modell felépítése

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, Embedding
from tensorflow.keras.optimizers import SGD

model = Sequential() 
model.add(Conv2D(32, kernel_size=(5, 5), activation='relu', input_shape=(28,28,1,))) 
model.add(Conv2D(64, (5, 5), activation='tanh')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax')) 

print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 24, 24, 32)        832       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 20, 20, 64)        51264     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 6400)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               819328    
_________________________________________________________________
dense_7 (Dense)              (None, 10)                1290      
Total params: 872,714
Trainable params: 872,714
Non-trainable params: 0
________________________________________________

# EarlyStopping, best modell, tanítás

In [26]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
patience=4
early_stopping=EarlyStopping(patience=patience, verbose=1)
checkpointer=ModelCheckpoint(filepath='model.hdf5', save_best_only=True, verbose=1)

In [28]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

network_history = model.fit(x_train, y_train, batch_size=50, epochs=10, verbose=1, validation_split=0.2, callbacks=[early_stopping, checkpointer])

Epoch 1/10
960/960 [==============================] - ETA: 0s - loss: 0.3547 - accuracy: 0.9030
Epoch 00001: val_loss improved from inf to 0.17432, saving model to model.hdf5
960/960 [==============================] - 166s 173ms/step - loss: 0.3547 - accuracy: 0.9030 - val_loss: 0.1743 - val_accuracy: 0.9501
Epoch 2/10
 69/960 [=>............................] - ETA: 2:23 - loss: 0.1561 - accuracy: 0.9533

KeyboardInterrupt: ignored

In [ ]:
from keras.models import load_model
model = load_model("model.hdf5")
test_err = model.evaluate(x_test,y_test)
print("Teszt hiba:", test_err[0], "Teszt pontosság:", test_err[1])

# Konfúziós mátrix + seaborn 

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, confusion_matrix
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred,1)
y_true = np.argmax(y_test,1)

print("test accuracy: %g" %(accuracy_score(y_true, y_pred)))
print("Precision", precision_score(y_true, y_pred, average="macro"))
print("Recall", recall_score(y_true, y_pred, average="macro"))
print("f1_score", f1_score(y_true, y_pred, average="macro"))
print("\nKonfúziós mátrix: ")
conf=confusion_matrix(y_true, y_pred)
print(conf)

In [ ]:
import seaborn as sns
sns.heatmap(conf, annot=True, fmt='d', vmax=20)